In [1]:
import os
import glob
from tqdm import tqdm

import pandas as pd

In [2]:
root = "/media/yesindeed/DATADRIVE1/mount/remote_cse/datasets/multitask-moe/rsna-str-pulmonary-embolism-detection"

df = pd.read_csv(f"{root}/train.csv")

df

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate
0,6897fa9de148,2bfbb7fd2e8b,c0f3cb036d06,0,0,0,0,0,0,1,1,0,0,1,0,0,0
1,6897fa9de148,2bfbb7fd2e8b,f57ffd3883b6,0,0,0,0,0,0,1,1,0,0,1,0,0,0
2,6897fa9de148,2bfbb7fd2e8b,41220fda34a3,0,0,0,0,0,0,1,1,0,0,1,0,0,0
3,6897fa9de148,2bfbb7fd2e8b,13b685b4b14f,0,0,0,0,0,0,1,1,0,0,1,0,0,0
4,6897fa9de148,2bfbb7fd2e8b,be0b7524ffb4,0,0,0,0,0,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790589,4833c9b6a5d0,57e3e3c5f910,da0ecef50cf5,0,0,0,0,0,0,1,1,1,0,0,0,0,0
1790590,4833c9b6a5d0,57e3e3c5f910,d74b46c2f2c4,0,0,0,0,0,0,1,1,1,0,0,0,0,0
1790591,4833c9b6a5d0,57e3e3c5f910,ba71189191ad,0,0,0,0,0,0,1,1,1,0,0,0,0,0
1790592,4833c9b6a5d0,57e3e3c5f910,f4fdc88f2ace,0,0,0,0,0,0,1,1,1,0,0,0,0,0


In [3]:
dict_out = {"id": [], "exam_id": [], "path": [], "class": [], "exam_class": []}

for i in range(len(df)):
    item = df.iloc[i]

    study_id = item["StudyInstanceUID"]
    series_id = item["SeriesInstanceUID"]
    sop_id = item["SOPInstanceUID"]

    img_file = f"{root}/train/{study_id}/{series_id}/{sop_id}.dcm"
    assert os.path.exists(img_file)

    dict_out["id"].append(f"{study_id}/{series_id}/{sop_id}")
    dict_out["exam_id"].append(f"{study_id}/{series_id}")

    dict_out["path"].append(f"./train/{study_id}/{series_id}/{sop_id}.dcm")

    cls = "no findings" if item["pe_present_on_image"] == 0 else "pulmonary embolism"
    exam_cls = "no findings" if item["negative_exam_for_pe"] == 1 else "pulmonary embolism"

    dict_out["class"].append(cls)
    dict_out["exam_class"].append(exam_cls)

df_out = pd.DataFrame.from_dict(dict_out)
df_out.head()

,id,exam_id,path,class,exam_class
0,6897fa9de148/2bfbb7fd2e8b/c0f3cb036d06,6897fa9de148/2bfbb7fd2e8b,./train/6897fa9de148/2bfbb7fd2e8b/c0f3cb036d06...,no findings,pulmonary embolism
1,6897fa9de148/2bfbb7fd2e8b/f57ffd3883b6,6897fa9de148/2bfbb7fd2e8b,./train/6897fa9de148/2bfbb7fd2e8b/f57ffd3883b6...,no findings,pulmonary embolism
2,6897fa9de148/2bfbb7fd2e8b/41220fda34a3,6897fa9de148/2bfbb7fd2e8b,./train/6897fa9de148/2bfbb7fd2e8b/41220fda34a3...,no findings,pulmonary embolism
3,6897fa9de148/2bfbb7fd2e8b/13b685b4b14f,6897fa9de148/2bfbb7fd2e8b,./train/6897fa9de148/2bfbb7fd2e8b/13b685b4b14f...,no findings,pulmonary embolism
4,6897fa9de148/2bfbb7fd2e8b/be0b7524ffb4,6897fa9de148/2bfbb7fd2e8b,./train/6897fa9de148/2bfbb7fd2e8b/be0b7524ffb4...,no findings,pulmonary embolism


In [4]:
df_out.to_csv(f"{root}/temp.csv", index=False)

In [5]:
import numpy as np

df_out["dataset"] = "rsna-pulmonary-embolism"
df_out["class_num"] = 2
df_out["contains_negative"] = 1
df_out["modality"] = "ct"

df_out = df_out.sample(frac=1).reset_index(drop=True)

exams = df_out["exam_id"].unique()
exam_size = len(exams)

test_indices = np.random.choice(
    list(range(exam_size)), int(exam_size * 0.15), replace=False)

test_exams = exams[test_indices]

df_out["split"] = "train"

df_out.loc[df_out["exam_id"].isin(test_exams), "split"] = "test"

In [6]:
df_out["split"].value_counts()

split
train    1518682
test      271912
Name: count, dtype: int64

In [7]:
df_out.to_csv(f"{root}/metadata.csv", index=False)